In [ ]:
#solo para Google Colab
!pip install geopandas

In [ ]:
#Librerías
import pandas as pd
import geopandas as gpd
import sys
import tqdm
sys.path.append(r"C:\Users\mfpen\Documents\Repositorios\geoloc2\preparacion_datos")
from funciones_cruces_limpieza import *
sys.path.append(r"C:\Users\mfpen\Documents\Repositorios\geoloc2\Valor_Catastral")
from Herramientas import cruzar_chinchetas_vs_dnue
sys.path.append(r'C:\Users\mfpen\Documents\Repositorios\geoloc2\Detecciondeterrenos')
from codigos import obtener_curt

# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#Rutas de bases
# Usar el de casas original
path_casas =r"H:\.shortcut-targets-by-id\19j98C-PJkrk3gkgU7V03dLiJ4Gn-hnNe\geoshaps\Atlacomulco\Rotacion_Atlacomulco\prueba_deteccion_3_rotacion.shp"
#CURT Municipal
path_CURT =r"D:\Secretaria\Bases_shp_INEGI\CURT\024 Atlacomulco\ATLACOMULCO_PREDIOS.shp"
#Industias filtro
path_industrias = r"H:\.shortcut-targets-by-id\19j98C-PJkrk3gkgU7V03dLiJ4Gn-hnNe\geoshaps\Atlacomulco\industria\Atlaco_industrias_filtro.shp"
#Usar chinchetas vs denue o la original
path_chinchetas = r"H:\.shortcut-targets-by-id\19j98C-PJkrk3gkgU7V03dLiJ4Gn-hnNe\geoshaps\Atlacomulco\atlacomulco_3105\Salidas\Chinchetas_atlacomulco\Chinchetas_atlacomulco_limpio.shp"
# Usar DENUE_EDOMEX_2022 en Estatal de Geoshaps o en Geo -> Data -> Bases_shp_INEGI -> DENUE -> (Municipio.shape)
path_denue =r"I:\Mi unidad\Cruces_predial_otros\DENUE\Atlacomulco.shp"
#Usar base de zonas rurales y urbanas
path_zonas = r"H:\.shortcut-targets-by-id\1xy7wVmj8pxJHqeyV9CG1vOBQs3emChdq\Geo\Data\bases_shp_INEGI\zonas_rurales_urbanas\15m.shp"



In [ ]:
#Carga de bases
casas = gpd.read_file(path_casas)
CURT_mun = gpd.read_file(path_CURT)
industrias=gpd.read_file(path_industrias)
pines = gpd.read_file(path_chinchetas)
denue = gpd.read_file(path_denue)
zonas = gpd.read_file(path_zonas)

Cruce de CURT x predicción de casas

In [ ]:
#Revisar que corresponda la base de casas únicamente a casas
casas.clase_dete.value_counts()

In [ ]:
#Filtrar para que no contenga las predicciones de terreno_baldio
casas=casas[casas.clase_dete != 'terreno_baldio']
casas

In [ ]:
#Se le asigna un ID 
casas["ID_casas"]=casas.index
casas

In [ ]:
#Se revisa el CRS de cada base para poder hacer el cruce entre ellas
print("crs CURT: ",CURT_mun.crs)
print("crs casas: ",casas.crs)

In [ ]:
#Configuración de la geometría para poder realizar el cruce x predicción
CURT_mun=CURT_mun.set_crs(6364,allow_override=True)
CURT_mun=CURT_mun.to_crs(3857)
CURT_mun.geometry

In [ ]:
#Configuración de la geometría para poder realizar el cruce x curt
casas=casas.set_crs(3857,allow_override=True)
casas.geometry

In [ ]:
#Se crea la variable CURT_f 
casas=obtener_curt(casas,'geometry')
casas

In [ ]:
#Se borran duplicados a partir de la CURT generada
casas.drop_duplicates(subset='CURT_f',inplace=True)

In [ ]:
CURT_mun.shape, casas.shape

In [ ]:
cruce_curt=gpd.sjoin(CURT_mun,casas)
cruce_curt#.shape

In [ ]:
len(cruce_curt.geometry.unique())

In [ ]:
#Identificador de los que sí cruzan x curt
cruce_curt['CURT_si']=1
cruce_curt.tail(3)

In [ ]:
#Base de los que no cruzan x CURT
no_cruzanxcurt=CURT_mun[~(CURT_mun.geometry.isin(cruce_curt['geometry']))]
no_cruzanxcurt

In [ ]:
#Identificador de los que no cruzan x curt
no_cruzanxcurt['CURT_si']=0
no_cruzanxcurt

In [ ]:
cruce_curt.shape, no_cruzanxcurt.shape

In [ ]:
#Se concatenan las bases de los que sí cruzan x curt y los que no
cruce_final_curt=pd.concat([cruce_curt,no_cruzanxcurt])
cruce_final_curt

In [ ]:
#Para detectar duplicados x geometría
geo_dupli=cruce_final_curt.geometry.value_counts()[cruce_final_curt.geometry.value_counts()>1].index
cruce_final_curt.loc[cruce_final_curt.geometry.isin(geo_dupli),"dupGEO"]=1
cruce_final_curt.loc[cruce_final_curt.dupGEO != 1, "dupGEO"]=0
cruce_final_curt

In [ ]:
#Para detectar duplicados x curt
curt_dupli=cruce_final_curt.curt.value_counts()[cruce_final_curt.curt.value_counts()>1].index
cruce_final_curt.loc[cruce_final_curt.curt.isin(curt_dupli),"dupCURT"]=1
cruce_final_curt.loc[cruce_final_curt.dupCURT != 1, "dupCURT"]=0
cruce_final_curt

In [ ]:
#Se crea la base de las predicciones de casas que no cruzan x CURT
casas_nocruzan = casas[~casas.ID_casas.isin(cruce_curt.ID_casas.unique())]
casas_nocruzan['YOLO_si'] = 1
casas_nocruzan.tail(2)

In [ ]:
cruce_final_curt.shape, casas_nocruzan.shape

In [ ]:
#Se concatenan las bases del cruce con la de las predicciones de casas que no cruzaron
cruce_final =pd.concat([cruce_final_curt,casas_nocruzan])
cruce_final

In [ ]:
#Identificador de los registros que no pertenecen a la predicción(CURT) y los que Sí cruzaron con CURT
cruce_final.loc[cruce_final.YOLO_si.isna(),"YOLO_si"]=0

In [ ]:
cruce_final.rename(columns={'index_right':'ID_curt'},inplace=True)

In [ ]:
cruce_final.rename(columns={'id_cat':'CLAVECATASTRAL'},inplace=True)

In [ ]:
cruce_final.to_file(r"I:\Mi unidad\Cruces_predial_otros\Atlacomulco\Bases_salida_limpieza\casas\NUEVO/Atlacomulco_casasxcurt.shp")

Cruce de predicción de casas vs Industrias

In [ ]:
cruce_final.shape, industrias.shape

In [ ]:
cruce_casasxind = gpd.sjoin(industrias,cruce_final)
cruce_casasxind.shape

In [ ]:
no_cruzanxInd=industrias[~(industrias.geometry.isin(cruce_casasxind['geometry']))]
no_cruzanxInd.tail(2)

In [ ]:
no_cruzanxInd['Ind_si']=0
no_cruzanxInd.tail(2)

In [ ]:
cruce_casasxind.shape, no_cruzanxInd.shape

In [ ]:
crucexInd=pd.concat([cruce_casasxind,no_cruzanxInd])
crucexInd

In [ ]:
nocruzan_casas=cruce_final[~cruce_final.ID_casas.isin(cruce_casasxind.ID_casas.unique())]
nocruzan_casas['Ind_si']=0
nocruzan_casas.tail(2)

In [ ]:
cruce_final_ind =pd.concat([crucexInd,nocruzan_casas])
cruce_final_ind

In [ ]:
cruce_final_ind.rename(columns={"index_right":"ID_ind"},inplace=True)
cruce_final_ind.columns

In [ ]:
cruce_final_ind.to_file(r"I:\Mi unidad\Cruces_predial_otros\Atlacomulco\Bases_salida_limpieza\casas\NUEVO/Atlacomulco_casas_Indxcurt.shp")

Cruce de chinchetas vs denue

In [ ]:
#SOLO CORRER SI ESTÁS EN COLAB, sino NO
def cruzar_chinchetas_vs_dnue(path_chinchetas,path_dnue,municipio=None,dist_max=10, 
                              schema_crs={'set_ch':3857,'to_ch':3857,
                                        'set_dnue':6364,'to_dnue':3857}, 
                              show_crs=False, show_results=True):
    '''
    (Function)
        Esta funcion cruza y agrupa los posibles nombres que se obtienen al poligono
        de una chincheta con relacion a la informacion extraida en dnue.
    (Parameters)
        - path_chinchetas: [str|GeoDataFrame] relativo a las chinchetas
        - path_dnue: [str|GeoDataFrame] relativo a dnue
        - municipio: [str] Nombre del municipio que desea analizar en relacion a dnue opcional
                    se considera para ver la numeralia con show_results
        - dist_max: [int|float] distancia maxima para considerar en el cruce, en relacion con schema_crs "to_ch and to_dnue"
        - schema_crs: Respete las llaves, en caso de abrir el shape, setea el shape con el valor de set,
                     y el to_ es para hacer una reproyecciones, estos deben ser iguales
        - show_crs: [bool] True para ver el crs que tiene cada shape
        - show_results: [bool] True para ver la numeralia del cruce, tendra mas sentido si pone el muncipio
    '''
    if isinstance(path_chinchetas,str):
        # Cargamos chinchetas
        gdf_ch = gpd.read_file(path_chinchetas)
        # print(gdf_ch.crs)
        gdf_ch = gdf_ch.set_crs(schema_crs['set_ch'],allow_override=True)
    elif isinstance(path_chinchetas,gpd.GeoDataFrame):
        gdf_ch = path_chinchetas
    else:
        raise ValueError('Definio mal path_chinchetas, debe ser un GeoDataFrame o bien la ruta al shape')
    gdf_ch = gdf_ch.to_crs(schema_crs['to_ch'])

    if isinstance(path_dnue,str):
        # Cargamos dnue
        gdf_dnue = gpd.read_file(path_dnue)
        # Seteamos el crs en caso de leerlo
        gdf_dnue   = gdf_dnue.set_crs(schema_crs['set_dnue'],allow_override=True)
    elif isinstance(path_dnue,gpd.GeoDataFrame):
        gdf_dnue = path_dnue
    else:
        raise ValueError('Definio mal path_dnue, debe ser un GeoDataFrame o bien la ruta al shape')
    gdf_dnue = gdf_dnue.to_crs(schema_crs['to_dnue'])

    if show_crs:
        print('CRS chinchetas: ',gdf_ch.crs)
        print('CRS Dnue      : ',gdf_dnue.crs)
    
    # Filtro
    if municipio != None:
        gdf_dnue = gdf_dnue[gdf_dnue['municipio']==municipio]

    # Primer cruce por interseccion
    cruce1 = gpd.sjoin(gdf_ch, gdf_dnue,
          how='left',)
    
    # Ids encontrados
    ids_find = cruce1[cruce1['id'].fillna('vacio')!='vacio'].id.astype(float).unique()
    ids_find = [int(x)  for x in ids_find]

    # Hacemos todo str menos la geometry para agrupar
    cruce1.fillna('vacio', inplace=True)
    for col in cruce1:
        if col == 'geometry':
            print('omitio ', col)
            continue
        cruce1[col] = cruce1[col].astype(str)

    '''
    Luego de hacer un primer cruce, es razonable pensar que un id, este en 1 o mas poligonos de chinchetas, las razones pueden ser; <br>
    - Es una plaza <br>
    - Es un mercado <br>
    - La separacion entre poligonos es minima <br>
    - otra <br>

    Entonces compactaremos todos esos id, en un solo registro de manere que concatenemos los posibles id que se corresponden a esa chincheta.
    '''
    # Agrupamos para concatenar los datos duplicados de cada geometria
    #print('Antes de cruce')
    cruce1 = pd.DataFrame(cruce1)
    cruce1_ = cruce1.groupby('geometry', as_index=False,sort=False).agg({'tipoCenCom': '|'.join, 'tipo_asent': '|'.join,
                                    'nombre_act': '|'.join,
                                    'id': '|'.join, 'clee': '|'.join})#.reset_index(drop=True)
    # Hacemos GeoDataFrame
    #print('Postcruce')
    cruce1_ = gpd.GeoDataFrame(cruce1_, geometry='geometry')

    # Asignamos lo encontrado al de chinchetas

    for i in tqdm.tqdm(cruce1_[cruce1_['id'].fillna('vacio')!='vacio'].index):
        gdf_ch.loc[gdf_ch['geometry'] == cruce1_.loc[i,'geometry'],'tipoCenCom'] = cruce1_.loc[i,'tipoCenCom']
        gdf_ch.loc[gdf_ch['geometry'] == cruce1_.loc[i,'geometry'],'tipo_asent'] = cruce1_.loc[i,'tipo_asent']
        gdf_ch.loc[gdf_ch['geometry'] == cruce1_.loc[i,'geometry'],'nombre_act'] = cruce1_.loc[i,'nombre_act']
        gdf_ch.loc[gdf_ch['geometry'] == cruce1_.loc[i,'geometry'],'clee'] = cruce1_.loc[i,'clee']
        gdf_ch.loc[gdf_ch['geometry'] == cruce1_.loc[i,'geometry'],'id']   = cruce1_.loc[i,'id']
    
    # Buscamos los que no hicieron match de chinchetas y de dnue
    falta_dnue = gdf_dnue[~gdf_dnue.id.isin(ids_find)]
    falta_ch = gdf_ch[gdf_ch.id.fillna('vacio')=='vacio']

    # Hacemos un segundo cruce por vecinos cercanos tomando distancia maxima
    cruce2 = gpd.sjoin_nearest(falta_ch[['Clase', 'geometry']], falta_dnue,
          how='left',max_distance=dist_max,distance_col='distancias',)
    
    # Hacemos todo str menos la geometry para agrupar
    cruce2.fillna('vacio', inplace=True)
    for col in cruce2:
        if col == 'geometry':
            print('omitio ', col)
            continue
        cruce2[col] = cruce2[col].astype(str)
    
    # Agrupamos para concatenar los datos duplicados de cada geometria
    cruce2_ = cruce2.groupby('geometry', as_index=False,sort=False).agg({'tipoCenCom': '|'.join, 'tipo_asent': '|'.join,
                                    'nombre_act': '|'.join,
                                    'id': '|'.join, 'clee': '|'.join}).reset_index(drop=True)
    # Hacemos GeoDataFrame
    cruce2_ = gpd.GeoDataFrame(cruce2_, geometry='geometry')

    # Asignamos los valores encontrados
    for i in tqdm.tqdm(cruce2_[cruce2_['id'].fillna('vacio')!='vacio'].index):
        gdf_ch.loc[gdf_ch['geometry'] == cruce2_.loc[i,'geometry'],'tipoCenCom'] = cruce2_.loc[i,'tipoCenCom']
        gdf_ch.loc[gdf_ch['geometry'] == cruce2_.loc[i,'geometry'],'tipo_asent'] = cruce2_.loc[i,'tipo_asent']
        gdf_ch.loc[gdf_ch['geometry'] == cruce2_.loc[i,'geometry'],'nombre_act'] = cruce2_.loc[i,'nombre_act']
        gdf_ch.loc[gdf_ch['geometry'] == cruce2_.loc[i,'geometry'],'clee']       = cruce2_.loc[i,'clee']
        gdf_ch.loc[gdf_ch['geometry'] == cruce2_.loc[i,'geometry'],'id']         = cruce2_.loc[i,'id']

    # Encontrados en el segundo cruce
    ids_find2 = cruce2[cruce2['id'].fillna('vacio')!='vacio'].id.astype(float).unique()
    ids_find2 = [int(x)  for x in ids_find2]

    if show_results:
        print('\nDnue ids encontrados    : ', len(ids_find2)+len(ids_find))
        print('Dnue faltan por asignar : ',gdf_dnue.shape[0]-len(ids_find)-len(ids_find2))
        print('Dnue total de id en dnue: ', gdf_dnue.shape[0])
        print('Chinchetas se encontraron     : ', gdf_ch[gdf_ch.id.fillna('vacio')!='vacio'].shape[0])
        print('Chinchetas falta por encontrar: ', gdf_ch[gdf_ch.id.fillna('vacio')=='vacio'].shape[0])
        print('Total chichentas              : ', gdf_ch.shape[0])
    return gdf_ch
def choose_city(gdf_zonas):
    '''
        Función para localizar el municipio a tratar dentro del shape de zonas rurales y urbanas
    '''
    zona = input('Digita el código de la zona que quieres (Ej. Ixtapan = 040):')
    gdf_mun = gdf_zonas[gdf_zonas.CVE_MUN == zona]
    #gdf_mun.plot()
    return gdf_mun
    
def cruce_limpieza_shapes(gdf_zonas,gdf_construcciones):
    gdf_mun = choose_city(gdf_zonas)
    gdf_mun = gdf_mun.to_crs(3857)
    gdf_cruce= gdf_construcciones.sjoin(gdf_mun,how='left')
    
    return gdf_cruce

In [ ]:
pines.columns,denue.columns

In [ ]:
#Eliminar las columnas que traen el concatenado de CLase y solo dejar la columna buena, en caso de que existan más de 1 de Clase
pines.drop(['Clase1','Clase'],axis=1,inplace=True)
pines.tail(2)

In [ ]:
#La columna de clasificacion de chinchetas debe llamarse Clase, sino cambiar
pines.rename(columns={'Clase_lim':'Clase'},inplace=True)
pines.tail(2)

In [ ]:
#En caso de tener que hacer el cruce de pines vs denue, ejecutar esta celda antes del cruce

schema_crs={'set_ch':3857,'to_ch':3857,'set_dnue':6364,'to_dnue':3857}
pines = pines.set_crs(schema_crs['set_ch'],allow_override=True)
denue   = denue.set_crs(schema_crs['set_dnue'],allow_override=True)
pines.set_geometry('geometry',inplace=True)

In [ ]:
#CRUCE pines vs denue
cruce_pinesxdenue=cruzar_chinchetas_vs_dnue(pines,denue)

In [ ]:
cruce_pinesxdenue.columns

In [ ]:
cruce_pinesxdenue.rename(columns={'id':'ID_denue','conf':'conf_pin','area':'area_pin'},inplace=True)

In [ ]:
cruce_pinesxdenue.drop(['clase_dete'],axis=1,inplace=True)

In [ ]:
#Guardar el archivo de la siguiente forma: Municipio_casas_pinesxdenue
cruce_pinesxdenue.to_file(r"I:\Mi unidad\Cruces_predial_otros\Atlacomulco\Bases_salida_limpieza\casas\NUEVO/Atlacomulco_casas_pinesxdenue.shp")

In [ ]:
#En el caso de ya tener pines vs denue, cambiar el nombre de la columna id a ID_denue y otras variables para no confundirlas con las de yolo
pines.rename(columns={'id':'ID_denue','conf':'conf_pin','area':'area_pin',"index_right":"ID_chin"},inplace=True)
pines.tail(2)
cruce_pinesxdenue=pines

In [ ]:
cruce_pinesxdenue.columns

Cruce de casas x anterior 

In [ ]:
cruce_final_ind.shape, cruce_pinesxdenue.shape

In [ ]:
cruce_casasxpines=gpd.sjoin(cruce_final_ind,cruce_pinesxdenue, how='left')
cruce_casasxpines

In [ ]:
cruce_casasxpines.columns

In [ ]:
#Filtrar las detecciones de casas que cruzan con chinchetas, buscar la columna correspondiente a la deteccion de casas de alexnet de la base de deteccion
cruce_casasxpines.loc[cruce_casasxpines.Clase.notna(),"clase_dete"] = 'establecimiento'

In [ ]:
#Filtrar las detecciones de casas que cruzan con chinchetas, buacar la columna correspondiente a la deteccion de casas de alexnet de la base de deteccion
cruce_casasxpines.loc[(cruce_casasxpines['clase_dete'] == 'establecimiento') & (cruce_casasxpines.Clase.isna()),"clase_dete"] = 'casas'

In [ ]:
#Revisar si la base de casas final tiene un ID_casas, si NO lo tiene generar
print('Columas de casas: ', casas.columns)
if input('Quiere crear la columna ID_casas') in ['1','si']:
    cruce_final_ind.reset_index(drop=False, inplace=True)
    cruce_final_ind.rename(columns={'index':'ID_casas'},inplace=True)

In [ ]:
cruce_casasxpines.columns

Cruce para identificación de zonas rurales y urbanas

In [ ]:
cruce_casasxpines.shape, zonas.shape

In [ ]:
#Visualización de base de zonas
zonas.tail(2)

In [ ]:
#Se filtra x la clave del municipio que se está trabajando
zona=zonas[zonas.CVE_MUN=='014']
zona.plot()

Cruce de zonas vs construcciones

In [ ]:
#Base del cruce con areas rurales y urbanas   casas vs 15m
base_ur = gpd.sjoin(cruce_casasxpines,zona.to_crs(3857),how='left')
base_ur.rename(columns={'index_right':'ID_15m'},inplace=True)
base_ur.shape, cruce_casasxpines.shape

In [ ]:
len(base_ur.geometry.unique())

In [ ]:
base_ur[base_ur.geometry.isin(base_ur.geometry.value_counts()[base_ur.geometry.value_counts()>1].index)].tail(2)

In [ ]:
base_ur.columns

In [ ]:
# Ejes de calles
ejes_path =r"H:\.shortcut-targets-by-id\1xy7wVmj8pxJHqeyV9CG1vOBQs3emChdq\Geo\Data\DELEGACIONES_EDOMEX\ATLACOMULCO_2018\ATLACOMULCO\Ejes_Atlacomulco.shp"
ejes = gpd.read_file(ejes_path)
ejes = ejes.to_crs(3857)
ejes.tail(2)

In [ ]:
cruce_ejes = base_ur.sjoin(ejes)
print(cruce_ejes.shape)
'Porcentaje a borrar: ',cruce_ejes.shape[0]/base_ur.shape[0]*100

In [ ]:
import numpy as np 
print('Shape original: ', base_ur.shape)
index_drop = np.unique(base_ur[base_ur.geometry.isin(cruce_ejes.geometry)].index.values)
base_ur.drop(index=index_drop,inplace=True)
print('Shape final:    ', base_ur.shape)

In [ ]:
base_ur.columns

In [ ]:
try:
    base_ur = base_ur[['ID_casas','clase_dete','Clase','geometry','CLAVECATASTRAL', 'curt','CURT_f',
                        'puntos','zona','area_left', 'rectangulo', 'Medida_lx', 'Medida_ly','Max_xy',
                        'Min_xy', 'f_lado', 'f_area',  'ID_ind', 'cve_ent','cve_mun', 'cve_loc',  'notas', 
                        'ID_curt','conf', 'area_right', 'status','CURT_si', 'dupGEO', 'dupCURT','YOLO_si', 
                        'area', 'Ind_si', 'ID_chin','conf_pin', 'area_pin','angulo',  'umbral', 'n_resize',
                        'tipoCenCom', 'tipo_asent', 'nombre_act','clee', 'ID_denue', 'ID_15m','CVEGEO',
                        'CVE_ENT', 'CVE_MUN', 'CVE_LOC','CVE_AGEB', 'CVE_MZA','AMBITO', 'TIPOMZA']]
except:
   base_ur = base_ur[['ID_casas','clase_dete','Clase','geometry','CLAVECATASTRAL', 'curt','CURT_f',
                        'puntos','zona','area_left', 'rectangulo', 'Medida_lx', 'Medida_ly','Max_xy',
                        'Min_xy', 'f_lado', 'f_area',  'ID_ind', 'cve_ent','cve_mun', 'cve_loc',  'notas', 
                        'ID_curt','conf', 'area_right', 'status','CURT_si', 'dupGEO', 'dupCURT','YOLO_si', 
                        'area', 'Ind_si', 'ID_chin','conf_pin', 'area_pin','angulo',  'umbral', 'n_resize',
                        'tipoCenCom', 'tipo_asent', 'nombre_act','clee', 'ID_denue', 'ID_15m','CVEGEO',
                        'CVE_ENT', 'CVE_MUN', 'CVE_LOC','CVE_AGEB', 'CVE_MZA','AMBITO', 'TIPOMZA']]

In [ ]:
base_ur.tail(3)

In [ ]:
base_ur.to_file(r'I:\Mi unidad\Cruces_predial_otros\Atlacomulco\Bases_salida_limpieza\casas\NUEVO/Atlacomulco_casas_cruces_todos.shp',
                 index=False)